In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_horizVert_wavelet_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,164691.75,0.83,1.05,-0.24,0.17,0.08,-0.01,-0.00
3,0.83,39108.39,-0.80,-0.11,0.06,-0.01,0.01,-0.00
4,1.05,-0.80,10821.33,-0.05,0.02,0.06,0.00,0.00
5,-0.24,-0.11,-0.05,2593.14,-0.03,-0.02,-0.00,-0.00
6,0.17,0.06,0.02,-0.03,493.77,0.00,0.00,0.00
7,0.08,-0.01,0.06,-0.02,0.00,79.43,-0.00,0.00
8,-0.01,0.01,0.00,-0.00,0.00,-0.00,11.14,0.00
9,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,1.02


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00001,0.00002,-0.00001,0.00002,0.00002,-0.00001,-0.00001
3,0.00001,1.00000,-0.00004,-0.00001,0.00001,-0.00000,0.00001,-0.00002
4,0.00002,-0.00004,1.00000,-0.00001,0.00001,0.00006,0.00000,0.00000
5,-0.00001,-0.00001,-0.00001,1.00000,-0.00002,-0.00005,-0.00001,-0.00000
6,0.00002,0.00001,0.00001,-0.00002,1.00000,0.00001,0.00001,0.00001
7,0.00002,-0.00000,0.00006,-0.00005,0.00001,1.00000,-0.00002,0.00006
8,-0.00001,0.00001,0.00000,-0.00001,0.00001,-0.00002,1.00000,0.00002
9,-0.00001,-0.00002,0.00000,-0.00000,0.00001,0.00006,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

2.255015563227933

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.39484940e+09 1.87515805e+08 1.39427446e+07 7.68382606e+05
 2.61368117e+04 6.02029143e+02 8.88960248e+00 6.80330475e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999311,-0.035846,-0.009321,-0.002226,-0.000423,-0.000068,-0.000010,-8.919559e-07
1,0.035308,0.998056,-0.050125,-0.010992,-0.002081,-0.000335,-0.000047,-4.421898e-06
2,0.010946,0.049119,0.997372,-0.051296,-0.009113,-0.001457,-0.000205,-1.881603e-05
3,0.003145,0.013286,0.050141,0.997377,-0.049785,-0.007661,-0.001072,-9.847450e-05
4,0.000745,0.003141,0.011372,0.048727,0.997185,-0.055247,-0.007483,-6.865289e-04
5,0.000159,0.000672,0.002421,0.010142,0.054109,0.995896,-0.071507,-6.363285e-03
6,0.000033,0.000141,0.000509,0.002127,0.011164,0.070097,0.993217,-9.206608e-02
7,0.000006,0.000026,0.000094,0.000393,0.002060,0.012807,0.091371,9.957323e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0006887035080954673,
 0.001944094407736463,
 0.0026279187644323576,
 0.0026228853360420468,
 0.002815350215855883,
 0.004103805825458662,
 0.006782596940830277,
 0.004267674224212925]